In [ ]:
!pip install mvlearn
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from mvlearn.semi_supervised import CTRegressor
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import itertools as iter
import warnings
warnings.filterwarnings("ignore")

In [ ]:
Dataset =pd.read_csv('Dataset.csv')
X=Dataset[['AT','V','AP','RH']]
y=Dataset[['PE']]
X_dash,X_test,y_dash,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
X_train,X_val,y_train,y_val=train_test_split(X_dash,y_dash,train_size=0.75,random_state=0)

In [ ]:
labeled_portion =0.001
y_train_semi=y_train.copy()
selector = np.random.uniform(size=(5740,))
selector[selector > labeled_portion] = np.nan
y_train_semi[np.isnan(selector)] = np.nan
lab_samples = ~np.isnan(y_train_semi)
not_null = [i for i in y_train_semi.index if not np.isnan(y_train_semi['PE'][i])]

#using all features and only labeled data
knn1 = KNeighborsRegressor(p=2)
knn2 = KNeighborsRegressor(p=5)

# traing with labeled samples only
knn1.fit(X_train.loc[not_null,['AT','V','AP','RH']], y_train_semi['PE'][not_null])
pred1 = knn1.predict(X_val[['AT','V','AP','RH']])
error1 = mean_squared_error(y_val, pred1)

knn2.fit(X_train.loc[not_null,['AT','V','AP','RH']], y_train_semi['PE'][not_null])
pred2 = knn2.predict(X_val[['AT','V','AP','RH']])
error2 = mean_squared_error(y_val, pred2)

print("MSE using only labeled data with knn1 {}\n".format(error1))
print("MSE using only labeled data with knn2 {}\n".format(error2))



# Multi-view co-training semi-supervised learning
estimator1 = KNeighborsRegressor(p=2)
estimator2 = KNeighborsRegressor(p=5)
knn = CTRegressor(estimator1, estimator2, random_state=26)

# Train a CTClassifier on all the labeled and unlabeled training data
knn.fit([X_train[['AT','AP']], X_train[['V','RH']]], y_train_semi)
pred_multi_view = knn.predict([X_val[['AT','AP']], X_val[['V','RH']]])
error_multi_view = mean_squared_error(y_val, pred_multi_view)

print(f"MSE of cotraining semi supervised regression {error_multi_view}")

print('MSE of trivial regressor',mean_squared_error(y_val,[np.mean(y_train_semi['PE'][not_null])]*len(y_val)))

In [ ]:
error_only_labeled_data=[]
semisupervised_error=[]
for j in np.arange(0.001,0.1,0.001):
    labeled_portion =j
    y_train_semi=y_train.copy()
    selector = np.random.uniform(size=(5740,))
    selector[selector > labeled_portion] = np.nan
    y_train_semi[np.isnan(selector)] = np.nan
    lab_samples = ~np.isnan(y_train_semi)
    not_null = [i for i in y_train_semi.index if not np.isnan(y_train_semi['PE'][i])]
    
    # Train single view on only the examples with labels
    knn2 = KNeighborsRegressor(p=5)
    knn2.fit(X_train.loc[not_null,['AT','V','AP','RH']], y_train_semi['PE'][not_null])
    pred2 = knn2.predict(X_val[['AT','V','AP','RH']])
    error2 = mean_squared_error(y_val, pred2)
    error_only_labeled_data.append(error2)
    
    
    
    
    # Multi-view co-training semi-supervised learning
    estimator1 = KNeighborsRegressor(p=2)
    estimator2 = KNeighborsRegressor(p=5)
    knn = CTRegressor(estimator1, estimator2, random_state=26)

    # Train a CTClassifier on all the labeled and unlabeled training data
    knn.fit([X_train[['AT','AP']], X_train[['V','RH']]], y_train_semi)
    pred_multi_view = knn.predict([X_val[['AT','AP']], X_val[['V','RH']]])
    error_multi_view = mean_squared_error(y_val, pred_multi_view)
    semisupervised_error.append(error_multi_view)
    


plt.plot(np.arange(0.001,0.1,0.001),error_only_labeled_data,label='error using only labeled data')
plt.plot(np.arange(0.001,0.1,0.001),semisupervised_error,label='error using labeled and unlabeled data')
plt.xlabel('number of labeled data')
plt.ylabel('errors')
plt.legend()
plt.show()

plt.plot(np.arange(0.001,0.1,0.001),error_only_labeled_data,label='error using only labeled data')
plt.plot(np.arange(0.001,0.1,0.001),semisupervised_error,label='error using labeled and unlabeled data')
plt.xlim(0,0.02)
plt.xlabel('number of labeled data')
plt.ylabel('errors')
plt.legend()
plt.show()